In this notebook I'll show how to maximize/minimize a function in Julia using the optim package. 

(We cover only unconstrained optimization with domain restrictions for now.)

# Minimizing a univariate function

We will find the minimum of the function $f(x)=(x-1)^2$. Admittedly, there is no need for using a computed to solve such a  simple problem but that makes it easier to show how to do it (and then you can use whatever complicated function you like).

We will use the Optim package. So, if you have not installed it yet, you need to install it now:

In [1]:
Pkg.add("Optim")

INFO: No packages to install, update or remove
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Optim
INFO: Use `Pkg.update()` to get the latest versions of your packages


Next I tell Julia to use Optim and define the function that I want to maximize.

In [10]:
using Optim

f(x)=(x-1)^2

f (generic function with 1 method)

and now we are ready for optimizing it: The command is "optimize" and its first argument is the function that you want  to optimize. Then you tell Optim a lower and an upper bound, i.e. you tell it an interval in which it whould look for the lowest function value. I choose -5 and 5 as boundaries which means that Optim will search for the x in the interval $[0,5]$ that minimizes $f$. A you can see, Optim finds the minimizer 1 and the minimum 0.

In [11]:
optimize(f,-5,5)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-5.000000, 5.000000]
 * Minimizer: 1.000000e+00
 * Minimum: 0.000000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6

## but I want to maximize...

If you want to maximize a function, say $g(x)=-(x-1)^2$, then you can just as well minimize the negative of the function; i.e. to solve $\max_x\, g(x)$ you can solve $\min_x\, -g(x)$. The minimizer is the maximizer in the original problem and the minimum is the negative of the maximum of the original problem. 

# Multivariate functions

First I have to tell you how to input multivariate functions into julia. There are several ways. I give several ways that all create the function 
$$g(z,y)=(1 - z)^2 + 100 * (y - z^2)^2$$

In [23]:
#option 1: math way
g(z,y)=(1 - z)^2 + 100 * (y - z^2)^2

#option 2: computer science way
function g(z,y)
    return (1 - z)^2 + 100 * (y - z^2)^2
end

g (generic function with 1 method)

There are further options: Instead of representing $g$ as a function of 2 variables I could represent it as a function of one variable $x$ where this variable $x$ is a vector with two components x[1] and x[2] which I do next.

(Strangely enough this is the representation that we will need for the minimization below.)

In [22]:
h(x)=(1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2

function h(x)
    return (1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2
end

h (generic function with 1 method)

Now we can minimize this multivariate function, i.e. solve
$$\min_{y,z} (1 - z)^2 + 100 * (y - z^2)^2.$$
The way we represent this problem in Optim is as minimization of one varaible $x$ which is a vector containing two variables (what used to be z and y above) which we call x[1], the first component of the vector, and x[2], the second component of the vector. The minimization problem looks then like
$$\min_x  (1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2.$$
Let's ask Optim to solve for the minimum. 

Unfortunately, some things change a bit compared to the univariate case above. Instead of telling optim an interval in which it should search, I am giving it a start point [0.0,0.0]. Note that you should write [0.0, 0.0] and not [0,0]. If you do the latter, Julia is not sure whether you want to consider integers only (I think...anyway you get an error if you try).

In [16]:
optimize(h,[0.0,0.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999634355313174,0.9999315506115275]
 * Minimum: 3.525527e-09
 * Iterations: 60
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 117

## but I need to give these box constraints

You might have a problem where variables are constraint, e.g. your variable represents a probability which can only be between 0 and 1. You can give these so called box-constraints to Optim also in the multivariate case. Say we need in the optimization problem above x[1] to be between 0 and 0.5 and x[2] between -2 and 1. We write this in the following way: After the starting value [0.0,0.0], we give the lower bounds [-0.5,-2.0], then the upper bounds [0.5,1.0] and finally we tell Optim that we want to use a method that can handle box contraints. This method is called Fminbox and because FminBox is itself a function we have to write it as "Fminbox()".

In [21]:
optimize(h,[0.0,0.0],[-0.5,-2.0],[0.5,1.0],Fminbox())

Results of Optimization Algorithm
 * Algorithm: Fminbox with Conjugate Gradient
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.49999999999963624,0.2500000000001418]
 * Minimum: 2.500000e-01
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 7.05e-10 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
     |f(x) - f(x')| / |f(x)| = 1.44e-09 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 9.99e-01 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 224
 * Gradient Calls: 172